# Use examples of [edges](https://github.com/romainsacchi/edges)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `edge` to use exchange-specific
characterization factors in the characterization matrix of `bw2calc`, combining the use of exchange names and locations.

## Requirements

* **Pyhton 3.10 or higher (up to 3.11) is highly recommended**

# Use case with [brightway2](https://brightway.dev/)

`brightway2` is an open source LCA framework for Python.
To use `premise` from `brightway2`, it requires that you have an activated `brightway2` project with a `biosphere3` database as well as an [ecoinvent](https://ecoinvent.prg) v.3 cut-off or consequential database registered in that project. Please refer to the brightway [documentation](https://brightway.dev) if you do not know how to create a project and install ecoinvent.

In [1]:
from edges import EdgeLCIA, get_available_methods
import bw2data

One can simply build its own LCIA file.
Let's consider the following LCIA file (saved under `lcia_example_2.json`):

In [2]:
[
  {
    "supplier": {
      "name": "Carbon dioxide",
      "operator": "startswith",
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process",
      "location": "CH" # now, we want to specify the location of the consumer
    },
    "value": 1.0
  },
  {
    "supplier": {
      "name": "Carbon dioxide",
      "operator": "startswith",
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process",
      "location": "FR" # now, we want to specify the location of the consumer
    },
    "value": 2.0
  },
  {
    "supplier": {
      "name": "Carbon dioxide",
      "operator": "startswith",
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process",
      "location": "RER" # now, we want to specify the location of the consumer
    },
    "value": 3.0
  }
]

[{'supplier': {'name': 'Carbon dioxide',
   'operator': 'startswith',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process', 'location': 'CH'},
  'value': 1.0},
 {'supplier': {'name': 'Carbon dioxide',
   'operator': 'startswith',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process', 'location': 'FR'},
  'value': 2.0},
 {'supplier': {'name': 'Carbon dioxide',
   'operator': 'startswith',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process', 'location': 'RER'},
  'value': 3.0}]

In [2]:
# activate the bw project
bw2data.projects.set_current("ecoinvent-3.10-cutoff")

In [3]:
act = bw2data.Database("ecoinvent-3.10.1-cutoff").random()
act

'market for electricity, medium voltage' (kilowatt hour, IL, None)

In [4]:
LCA = EdgeLCIA(
    demand={act: 1},
    method=("some", "method"),
    filepath="lcia_example_2.json"
)
LCA.lci()

LCA.map_exchanges()

LCA.evaluate_cfs()
LCA.lcia()
LCA.score

/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


Identifying eligible exchanges...


100%|██████████| 3/3 [00:00<00:00, 223.51it/s]


0.5471837461844041

### Generate dataframe of characterization factors used

The `generate_cf_table` method generates a dataframe of the characterization factors used in the calculation. One can see the characterization factors used for each exchange in the system.

In [5]:
df = LCA.generate_cf_table()

In [6]:
# we can see under the "CF" column
# the characterization factors used for each exchange in the system
df

,supplier name,supplier categories,consumer name,consumer reference product,consumer location,amount,CF,impact
0,"Carbon dioxide, fossil","(air, urban air close to ground)","treatment of sewage sludge, 70% water, WWT, WW...","sewage sludge, 70% water, WWT, WW from liquid ...",CH,7.057870e-17,1.0,7.057870e-17
1,"Carbon dioxide, fossil","(air, urban air close to ground)","treatment of sewage sludge, 70% water, WWT-SLF...","sewage sludge, 70% water, WWT-SLF, refinery sl...",CH,5.554115e-13,1.0,5.554115e-13
2,"Carbon dioxide, fossil","(air, urban air close to ground)","treatment of leachate, SLF, waste asphalt, was...","leachate, SLF, waste asphalt",CH,2.616844e-10,1.0,2.616844e-10
3,"Carbon dioxide, fossil","(air, urban air close to ground)","treatment of leachate, SLF, waste bitumen, was...","leachate, SLF, waste bitumen",CH,8.048970e-11,1.0,8.048970e-11
4,"Carbon dioxide, fossil","(air, urban air close to ground)","heat and power co-generation, natural gas, 500...","electricity, high voltage",CH,4.593825e-06,1.0,4.593825e-06
...,...,...,...,...,...,...,...,...
1207,"Carbon dioxide, non-fossil","(air,)","containerboard production, linerboard, kraftliner","containerboard, linerboard",RER,1.682536e-07,3.0,5.047607e-07
1208,"Carbon dioxide, non-fossil","(air,)",anthraquinone production,anthraquinone,RER,5.977578e-10,3.0,1.793273e-09
1209,"Carbon dioxide, non-fossil","(air,)",latex production,latex,RER,8.163367e-12,3.0,2.449010e-11
1210,"Carbon dioxide, non-fossil","(air,)","particleboard production, cement bonded","particleboard, cement bonded",RER,2.846972e-11,3.0,8.540915e-11


As expected, only CH, FR and RER-based consumers have been considered in the calculation.

In [7]:
df.groupby("consumer location")["CF"].mean()

consumer location
CH     1.0
FR     2.0
RER    3.0
Name: CF, dtype: float64

But `statistics()` tells us that many exchanges have been ignored.

In [8]:
LCA.statistics()

+----------------------+------------------------------------------+
|       Activity       | market for cleaning consumables, without |
|                      |      water, in 13.6% solution state      |
|     Method name      |            ('some', 'method')            |
|      Data file       |              lcia_example_2              |
| Unique CFs in method |                    3                     |
|   Unique CFs used    |                    3                     |
|  Exc. characterized  |                   1212                   |
| Exc. uncharacterized |                  334854                  |
+----------------------+------------------------------------------+


We can extend the application of CFs to other exchanges, using the helping functions: `map_aggregate_locations`, `map_disaggregate_location`, `map_dynamic_locations` and `map_remaining_locations_to_global`.

* `map_aggregate_locations` maps exchanges with aggregate locations (e.g., "RER", RLA", etc.) to weighted-average CFs.
* `map_disaggregate_location` maps exchanges with locations that belong to aggregate locations ("CA-QC") to CFs of aggregate locations (e.g., "CA").
* `map_dynamic_locations` maps exchanges with dynamic locations (e.g., "RoW", "RoW") to CFs calculated as the weighted-average of all locations minus those excluded by dynamic locations' perimeter (e.g., "GLO minus CH").
* `map_remaining_locations_to_global` maps exchanges with remaining locations to the global CFs.

In [4]:
LCA = EdgeLCIA(
    demand={act: 1},
    method=("some", "method"),
    filepath="lcia_example_2.json"
)
LCA.lci()

LCA.map_exchanges()

LCA.map_aggregate_locations()
LCA.map_dynamic_locations()
LCA.map_disaggregate_locations()
LCA.map_remaining_locations_to_global()

LCA.evaluate_cfs()
LCA.lcia()
LCA.score

/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


Identifying eligible exchanges...


100%|██████████| 3/3 [00:00<00:00, 238.70it/s]


Handling static regions...


100%|██████████| 334699/334699 [00:00<00:00, 350791.94it/s]
0it [00:00, ?it/s]


...

Handling unmatched locations...


100%|██████████| 334699/334699 [00:00<00:00, 341249.22it/s]
0it [00:00, ?it/s]


Handling remaining exchanges...


100%|██████████| 334699/334699 [00:00<00:00, 388285.88it/s]
0it [00:00, ?it/s]


0.0017014460518306926

In [5]:
LCA.statistics()

+----------------------+----------------------------------------+
|       Activity       | market for electricity, medium voltage |
|     Method name      |           ('some', 'method')           |
|      Data file       |             lcia_example_2             |
| Unique CFs in method |                   3                    |
|   Unique CFs used    |                   3                    |
|  Exc. characterized  |                  1210                  |
| Exc. uncharacterized |                 334699                 |
+----------------------+----------------------------------------+


In [6]:
df = LCA.generate_cf_table()

In [8]:
df

,supplier name,supplier categories,consumer name,consumer reference product,consumer location,amount,CF,impact
0,"Carbon dioxide, non-fossil","(air, urban air close to ground)","treatment of waste wood, untreated, municipal ...","waste wood, untreated",CH,4.124792e-06,1.0,4.124792e-06
1,"Carbon dioxide, non-fossil","(air, urban air close to ground)","biomethane, low pressure burned in micro gas t...","heat, central or small-scale, other than natur...",CH,8.083118e-20,1.0,8.083118e-20
2,"Carbon dioxide, non-fossil","(air, urban air close to ground)","treatment of digester sludge, municipal incine...","heat, for reuse in municipal waste incineratio...",CH,1.033653e-08,1.0,1.033653e-08
3,"Carbon dioxide, non-fossil","(air, urban air close to ground)","lath, hardwood, raw, kiln drying to u=20%","sawnwood, lath, hardwood, raw, dried (u=20%)",CH,3.437403e-08,1.0,3.437403e-08
4,"Carbon dioxide, non-fossil","(air, urban air close to ground)",beet sugar production,sugar beet pulp,CH,2.884184e-28,1.0,2.884184e-28
...,...,...,...,...,...,...,...,...
1205,"Carbon dioxide, non-fossil, resource correction","(natural resource, in air)","sulfate pulp production, from softwood, bleached","bark chips, wet, measured as dry mass",RER,1.066332e-09,3.0,3.198996e-09
1206,"Carbon dioxide, non-fossil, resource correction","(natural resource, in air)","particleboard production, cement bonded","particleboard, cement bonded",RER,1.816462e-08,3.0,5.449387e-08
1207,"Carbon dioxide, non-fossil, resource correction","(natural resource, in air)",three and five layered board production,"wood chips, dry, measured as dry mass",RER,1.565582e-07,3.0,4.696745e-07
1208,"Carbon dioxide, non-fossil, resource correction","(natural resource, in air)","sulfate pulp production, from softwood, unblea...","sawdust, wet, measured as dry mass",RER,3.875551e-11,3.0,1.162665e-10
